Importing the required packages

In [1]:
import spacy
import pdfminer
import re
import os
import pandas as pd

Imported the code inside the pdf2text file

In [2]:
import pdf2txt

Converting PDFs to text

In [3]:
def convert_pdf(f):
    output_filename = os.path.basename(os.path.splitext(f)[0]) + '.txt'
    output_filepath = os.path.join('output/txt/', output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath])
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

Loading the NLP language model

In [4]:
nlp = spacy.load('en_core_web_sm')

Defined a data structure which has the data fields needed

In [5]:
result_dict = {
    'name': [],
    'phone': [],
    'email': [],
    'skills': []
}

names = []
phones = []
emails = []
skills = []

Wrote a function that parses the content of a text file

In [7]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ == "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully")

Added a for loop so that the logic can work on multiple files instead of one file

In [9]:
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading '+ file)
        txt = convert_pdf(os.path.join('resumes/', file))
        parse_content(txt)

Reading AshleyMilesCV.pdf
output/txt/AshleyMilesCV.txt saved successfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully
Reading Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved successfully!!!
Alisson Parker-Wright                                                                                                 
alli1414parks@mail.com
Extraction completed successfully
Reading John DominicCV.pdf
output/txt/John DominicCV.txt saved successfully!!!
John Dominic
johndominic@mail.com
Extraction completed successfully


In [10]:
names

['Ashley Miles',
 'Alisson Parker-Wright                                                                                                 ',
 'John Dominic']

In [11]:
phones

["['6592251422']", "['8569878511']", "['7877756411']"]

In [12]:
emails

[ashleymiles@memail.com, alli1414parks@mail.com, johndominic@mail.com]

In [13]:
skills

["{'tableau', 'sql'}",
 "{'tableau', 'java', 'python'}",
 "{'java', 'python', 'hadoop'}"]

Assigning the individual lists to the result_dict dictionary

In [14]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skills'] = skills

Showing all the objects in the dictionary

In [15]:
result_dict

{'name': ['Ashley Miles',
  'Alisson Parker-Wright                                                                                                 ',
  'John Dominic'],
 'phone': ["['6592251422']", "['8569878511']", "['7877756411']"],
 'email': [ashleymiles@memail.com,
  alli1414parks@mail.com,
  johndominic@mail.com],
 'skills': ["{'tableau', 'sql'}",
  "{'tableau', 'java', 'python'}",
  "{'java', 'python', 'hadoop'}"]}

Rendering out the data in a table

In [16]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'tableau', 'sql'}"
1,Alisson Parker-Wright ...,['8569878511'],alli1414parks@mail.com,"{'tableau', 'java', 'python'}"
2,John Dominic,['7877756411'],johndominic@mail.com,"{'java', 'python', 'hadoop'}"
